In [21]:
import os
print(os.getcwd())

/teamspace/studios/this_studio/100-class-image-classifaction


In [22]:
# os.chdir("100-class-image-classifaction")

FileNotFoundError: [Errno 2] No such file or directory: '100-class-image-classifaction'

In [ ]:
!pwd

/teamspace/studios/this_studio/100-class-image-classifaction


In [23]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class BaseModelConfig:
    pretrained_model: str
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_include_top: bool
    params_weights: str
    params_image_size: list
    params_learning_rate: float
    params_optimizer: str
    params_dropout_rate: float
    params_classes: int

In [24]:
from LCIC import logger
from LCIC.constants import *
from LCIC.utils.common import read_yaml, create_directories

In [25]:
class BaseModelConfigurationManager:
    def __init__(self,
        model_name:str="",
        config_path: Path = CONFIG_FILE_PATH,
        params_path: Path = PARAMS_FILE_PATH):
        if model_name == "" or model_name is None:
            model_name = input("Enter the model name (all lower-case): ")
        self.model_name = model_name.upper() # for selecting different pretrained models
        self.config = read_yaml(config_path)
        # only the specified model configuration it will take.
        self.params = read_yaml(params_path)[self.model_name]

        create_directories([self.config.artifacts_root])
        logger.info(f"Preparing config manager for {self.model_name} pretrained model.")

    def get_base_model_config(self)->BaseModelConfig:
        base_model_cfg = self.config.base_model

        logger.info(f"Configurations ---> {base_model_cfg}")
        logger.info(f"Parameters ---> {self.params}")

        create_directories([base_model_cfg.root_dir])

        _cfg = BaseModelConfig(
            pretrained_model = self.model_name,
            root_dir = Path(base_model_cfg.root_dir),
            base_model_path= Path(base_model_cfg.base_model_path),
            updated_base_model_path= Path(base_model_cfg.updated_base_model_path),
            params_include_top= self.params.INCLUDE_TOP,
            params_weights= self.params.WEIGHTS,
            params_image_size= self.params.IMAGE_SIZE,
            params_optimizer= self.params.OPTIMIZER,
            params_learning_rate= self.params.LEARNING_RATE,
            params_dropout_rate= self.params.DROPOUT_RATE,
            params_classes= self.params.CLASSES  # need to look at.....!
        )
        return _cfg

In [26]:
import tensorflow as tf

In [27]:
class PrepareBaseModel:
    def __init__(self, config: BaseModelConfig):
        self.model= None
        self.config = config
        
    def get_base_model(self):
        if self.config.pretrained_model == "VGG16".upper():
            self.model = tf.keras.applications.vgg16.VGG16(
                input_shape=self.config.params_image_size,
                weights= self.config.params_weights,
                include_top= self.config.params_include_top
            )
        elif self.config.pretrained_model == "EfficientNetB2".upper():
            self.model = tf.keras.applications.efficientnet.EfficientNetB2(
                input_shape=self.config.params_image_size,
                weights= self.config.params_weights,
                include_top= self.config.params_include_top
            )
        
        elif self.config.pretrained_model == "EfficientNetB6".upper():
            self.model = tf.keras.applications.efficientnet.EfficientNetB2(
                input_shape=self.config.params_image_size,
                weights= self.config.params_weights,
                include_top= self.config.params_include_top
            )
        
        elif self.config.pretrained_model == "InceptionV3".upper():
            self.model = tf.keras.applications.inception_v3.InceptionV3(
                input_shape=self.config.params_image_size,
                weights= self.config.params_weights,
                include_top= self.config.params_include_top
            )
        elif self.config.pretrained_model == "ResNet101".upper():
            self.model = tf.keras.applications.resnet.ResNet101(
                input_shape=self.config.params_image_size,
                weights= self.config.params_weights,
                include_top= self.config.params_include_top
            )
        elif self.config.pretrained_model == "DenseNet169".upper():
            self.model = tf.keras.applications.densenet.DenseNet169(
                input_shape=self.config.params_image_size,
                weights= self.config.params_weights,
                include_top= self.config.params_include_top
            )
        
        self.save_model(path=self.config.base_model_path, model= self.model)
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        
    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, optimizer, learning_rate, dropout_rate):    
        # Freeze the pretrained weights
        if freeze_all:
            for layer in model.layers:
                layer.trainable= False
                
            no_of_layers = len(model.layers)
            no_of_layers_to_train = int(no_of_layers // 2)
    
            # Skipping the BN layers
            for layer in model.layers[-no_of_layers_to_train:]:
                if not isinstance(layer, tf.keras.layers.BatchNormalization):
                    layer.trainable = True
        
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model[:freeze_till]:
                if not isinstance(layer, tf.keras.layers.BatchNormalization):
                    layer.trainable = True
                else:
                    layer.trainable = False
        logger.info(f"Creating custome layers for the model")
        # Rebuild top
        x = tf.keras.layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dense(256, activation='relu')(x)
        x = tf.keras.layers.Dropout(dropout_rate, name="top_dropout")(x)
        x = tf.keras.layers.Dense(units=128, activation='relu')(x)
        
        prediction = tf.keras.layers.Dense(
            units= classes,
            activation='softmax',
            name="output_layer"
        )(x)
        
        full_model = tf.keras.models.Model(
            inputs= model.input,
            outputs = prediction
        )
        
        logger.info("Compileing the model")

        if optimizer.lower() == 'adam':
            full_model.compile(
                optimizer= tf.keras.optimizers.Adam(learning_rate= learning_rate),
                loss= tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy']
            )
        elif optimizer.lower() == 'sgd':
            full_model.compile(
                optimizer= tf.keras.optimizers.SGD(learning_rate= learning_rate, momentum=0.9),
                loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy']
            )
        elif optimizer.lower() == 'rmsprop':
            full_model.compile(
                optimizer= tf.keras.optimizers.RmsProp(learning_rate=learning_rate),
                loss= tf.keras.losses.CategoricalCrossentropy(),
                metrics=['accuracy']
            )
        else:
            raise ValueError("Undefine optimizer function")

        full_model.summary()
        logger.info(f"Loaded the base model.")
        return  full_model
    
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model = self.model,
            classes= self.config.params_classes,
            freeze_all= True,
            freeze_till= None,
            optimizer= self.config.params_optimizer,
            learning_rate= self.config.params_learning_rate,
            dropout_rate= self.config.params_dropout_rate
        )
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
        
        logger.info(f"Final model is saving at {self.config.updated_base_model_path}")

In [30]:
try:
    config_manager = BaseModelConfigurationManager()
    base_model_cfg_obj = config_manager.get_base_model_config()

    base_model_creator_obj = PrepareBaseModel(config=base_model_cfg_obj)
    base_model_creator_obj.get_base_model()
    base_model_creator_obj.update_base_model()
    
except Exception as e:
    logger.error("Error in creating base model")
    raise e

[2024-01-10 05:33:44,321]: INFO: common: yaml file: config/config.yaml loaded successfully!
[2024-01-10 05:33:44,327]: INFO: common: yaml file: params.yaml loaded successfully!
[2024-01-10 05:33:44,328]: INFO: common: Created directory at : artifacts
[2024-01-10 05:33:44,328]: INFO: 752381983: Preparing config manager for EFFICIENTNETB6 pretrained model.
[2024-01-10 05:33:44,329]: INFO: 752381983: Configurations ---> {'root_dir': 'artifacts/base_model', 'base_model_path': 'artifacts/base_model/base_model.keras', 'updated_base_model_path': 'artifacts/base_model/pretrained_model.keras'}
[2024-01-10 05:33:44,330]: INFO: 752381983: Parameters ---> {'AUGMENTATION': True, 'IMAGE_SIZE': [224, 224, 3], 'BATCH_SIZE': 16, 'INCLUDE_TOP': False, 'EPOCHS': 3, 'CLASSES': 4, 'WEIGHTS': 'imagenet', 'OPTIMIZER': 'Adam', 'LEARNING_RATE': 0.001, 'DROPOUT_RATE': 0.2}
[2024-01-10 05:33:44,331]: INFO: common: Created directory at : artifacts/base_model
31790344/31790344 [==============================] - 0s

In [29]:
"DenseNet169".upper()

'DENSENET169'